In [ ]:
import pandas as pd
import numpy as np
import json
import math
import urllib.request
import dateutil.parser
import dateutil.rrule
import dateutil.tz
import datetime
import re
import gc
from time import sleep

In [ ]:
tzUTC = dateutil.tz.gettz('UTC')
tzLocal = dateutil.tz.gettz('Europe/London')

# Used across all of the plots
dateToday = datetime.datetime.combine(datetime.date.today(), datetime.datetime.min.time()).replace(tzinfo=tzUTC)
baselineEnd = datetime.datetime.strptime('2020-03-16T23:59:59Z', '%Y-%m-%dT%H:%M:%SZ').replace(tzinfo=tzUTC)

resampleFrequency = 480

In [ ]:
# Doesn't do anything with old data yet, because of the large number of gaps in recent...
try:
    dfPointInterpTsOld = pd.read_pickle('../cache/recent-traffic-volumes-pd.pkl')
    dfPointInterpTsOld = dfPointInterpTsOld[dfPointInterpTsOld.index < dateToday - pd.Timedelta(days=2)]
    previousDataEnd = np.max(dfPointInterpTsOld.index).replace(tzinfo=tzLocal).astimezone(tzUTC)
    print('Loaded previous data.')
    print('  %s' % previousDataEnd)
except:
    dfPointInterpTsOld = None
    previousDataEnd = baselineEnd
    print('No existing data could be loaded.')

In [ ]:
# Identify all of the journey time pair links

print('Last updated %s' % (datetime.datetime.now(tzLocal).strftime('%d %B %Y %H:%M')))

anprRequestBase = 'https://api.newcastle.urbanobservatory.ac.uk/api/v2/sensors/entity'

# Fetch a list of all the car parks...
anprLinks = {}
anprRequestPage = 1
anprResponse = None

anprNameMatcher = re.compile('^(.*) - (.*) to (.*)$')

while anprResponse is None or len(anprResponse) > 1:
    anprResponse = json.loads(
        urllib.request.urlopen(
            '%s?name="Vehicle%%20monitoring%%20pair%%20"&page=%u' % (anprRequestBase, anprRequestPage)
        ).read().decode('utf-8')
    )['items']

    anprRequestPage = anprRequestPage + 1

    for journeyTimeLink in anprResponse:     
        for feed in journeyTimeLink['feed']:
            systemCodeNumber = feed['brokerage'][0]['sourceId'].split(':')[0]
            
            # Some links are set up to only count bus numberplates for public transport journey times
            if systemCodeNumber.endswith('_BUS') or \
               'latest' not in feed['timeseries'][0]:
                continue
            
            if not systemCodeNumber in anprLinks:
                linkDescription = journeyTimeLink['meta'].copy()
                linkDescription['timeseriesIRIs'] = {}
                anprLinks[systemCodeNumber] = linkDescription
                print('Discovered monitoring link "%s"' % anprLinks[systemCodeNumber]['longName'].strip())
            
            anprLinks[systemCodeNumber]['systemCodeNumber'] = systemCodeNumber
            
            for ts in feed['timeseries']:
                timeseriesType = None
                
                if feed['metric'] == 'Journey time':
                    timeseriesType = 'timeseriesJourneyTime'
                elif feed['metric'] == 'Number plates at start of link':
                    timeseriesType = 'timeseriesPlatesIn'
                elif feed['metric'] == 'Number plates at end of link':
                    timeseriesType = 'timeseriesPlatesOut'
                    
                for link in ts['links']:
                    if link['rel'] == 'archives' and timeseriesType is not None:
                        anprLinks[systemCodeNumber]['timeseriesIRIs'][timeseriesType] = link['href']
            
            nameElements = anprNameMatcher.match(anprLinks[systemCodeNumber]['longName'])

            if nameElements is None:
                print('Unable to match name "%s". Skipping.' % anprLinks[systemCodeNumber]['longName'])
                del anprLinks[systemCodeNumber]
                continue

            anprLinks[systemCodeNumber]['highwayDescription'] = nameElements[1]
            anprLinks[systemCodeNumber]['startDescription'] = nameElements[2]
            anprLinks[systemCodeNumber]['endDescription'] = nameElements[3]

anprLinks = pd.DataFrame.from_records(list(anprLinks.values()), index=['systemCodeNumber'])
print('Discovered %u ANPR pairs.' % len(anprLinks.index))

anprLinks

In [ ]:
# Convert links into a list of measurement points

anprPoints = {}
anprDirectionMatcher = re.compile('^.*\((.*)\)$')
anprDuplicateRegister = []

for systemCodeNumber in anprLinks.index:
    linkDefinition = anprLinks[anprLinks.index == systemCodeNumber]
    for end in ['start', 'end']:
        coordinates = (linkDefinition[end + 'Easting'].values[0], linkDefinition[end + 'Northing'].values[0])
        pointDescription = linkDefinition[end + 'Description'].values[0].strip()
        highwayDescription = linkDefinition['highwayDescription'].values[0].strip()
        countDirection = anprDirectionMatcher.match(linkDefinition['highwayDescription'].values[0].strip())
        vectorId = '%s (%s)' % (coordinates, countDirection)
        
        if countDirection is None:
            print('Unable to find direction in description "%s"' % highwayDescription)
            countDirection = 'Unknown'
        else:
            countDirection = countDirection[1]
        
        #print(coordinates, pointDescription, highwayDescription)
        
        if vectorId in anprPoints:
            anprPoints[vectorId]['linkCount'] = anprPoints[vectorId]['linkCount'] + 1
            anprDuplicateRegister.append({
                'originalId': systemCodeNumber,
                'originalEnd': end,
                'isAlias': True,
                'timeseriesName': anprPoints[vectorId]['timeseriesName']
            })
        else:
            vehicleCountName = '%s.%s.%s' % (systemCodeNumber, end, countDirection.lower())
            anprPoints[vectorId] = {
                'systemCodeNumber': systemCodeNumber,
                'end': end,
                'timeseriesName': vehicleCountName,
                'pointDescription': pointDescription,
                'highwayDescription': highwayDescription,
                'easting': linkDefinition[end + 'Easting'].values[0],
                'northing': linkDefinition[end + 'Northing'].values[0],
                'linkCount': 1
            }
            anprDuplicateRegister.append({
                'alternativeId': systemCodeNumber,
                'originalEnd': end,
                'isAlias': False,
                'timeseriesName': anprPoints[vectorId]['timeseriesName']
            })
            
anprPoints = pd.DataFrame.from_records(list(anprPoints.values()))

print('Found %u unique monitoring points.' % len(anprPoints.index))
anprPoints.head(40)

In [ ]:
pdAnprDuplicateRegister = pd.DataFrame.from_records(anprDuplicateRegister)
pdAnprDuplicateRegister

In [ ]:
dfLinkHistoric = pd.read_pickle('../cache/baseline-traffic-volumes-pd.pkl')
dfLinkHistoric.index = dfLinkHistoric.index.tz_localize(tzUTC).tz_convert(tzLocal)
dfLinkHistoric.sort_index(inplace=True)
gc.collect()

dfLinkHistoric.head(5)

In [ ]:
daysPerRequest = 30

dfPointTs = None

for pointIndex in anprPoints.index:
    pointRow = anprPoints[anprPoints.index == pointIndex]
    point = pointRow.to_dict(orient='records')[0]
    systemCodeNumber = point['systemCodeNumber']
    linkRow = anprLinks[anprLinks.index == systemCodeNumber]
    link = linkRow.to_dict(orient='records')[0]
    linkIRIs = link['timeseriesIRIs']
    
    #vehicleCountName = '%s at %s' % (point['highwayDescription'], point['pointDescription'])
    vehicleCountName = point['timeseriesName'] # '%s.%s' % (point['systemCodeNumber'], point['end'])
    vehicleCountIRIRequired = 'timeseriesPlatesIn' if point['end'] == 'start' else 'timeseriesPlatesOut'
    
    historicColumn = '%s.%s' % (
        systemCodeNumber,
        'platesIn' if point['end'] == 'start' else 'platesOut'
    )
    
    if vehicleCountIRIRequired not in linkIRIs:
        print('No data available for %s' % vehicleCountName)
        continue
    
    vehicleCountIRI = linkIRIs[vehicleCountIRIRequired]
    
    print(vehicleCountName)
    print('  [', end='')
    
    # TODO: Load the base data here instead
    pointTimeseries = None
    
    for windowStart in dateutil.rrule.rrule(
        dateutil.rrule.DAILY,
        interval=daysPerRequest,
        dtstart=previousDataEnd + pd.Timedelta(seconds=1),
        until=dateToday + pd.Timedelta(hours=24)
    ):
        windowEnd = windowStart + pd.Timedelta(days=daysPerRequest) - pd.Timedelta(seconds=1)

        if windowEnd > dateToday + pd.Timedelta(hours=24):
            windowEnd = dateToday + pd.Timedelta(hours=24)
            
        windowResponse = json.loads(
            urllib.request.urlopen(
              '%s?startTime=%s&endTime=%s' % (vehicleCountIRI, windowStart.isoformat().replace('+00:00', 'Z'), windowEnd.isoformat().replace('+00:00', 'Z'))
            ).read().decode('utf-8')
        )['historic']['values']

        if pointTimeseries is None:
            pointTimeseries = windowResponse
        else:
            pointTimeseries.extend(windowResponse)
        print('.', end='')
        
    print(']')
    
    if np.sum(list(map(lambda v: v['value'], pointTimeseries))) < 1:
        print('Empty timeseries returned for %s' % vehicleCountName)
        continue
    
    #if pointTimeseries is not None:
    #    break

    dfPoint = pd.DataFrame \
        .from_records(pointTimeseries, exclude=['duration']) \
        .rename(columns={'value': vehicleCountName})
    dfPoint['time'] = dfPoint['time'].apply(lambda t: datetime.datetime.strptime(t, "%Y-%m-%dT%H:%M:%S.%fZ").replace(tzinfo=tzUTC).astimezone(tzLocal))
    dfPoint.set_index('time', inplace=True, drop=True)
    
    if dfPointInterpTsOld is not None and vehicleCountName in dfPointInterpTsOld.columns:
        dfPoint = pd.concat([
            dfPointInterpTsOld[vehicleCountName],
            dfPoint[vehicleCountName]
        ]).to_frame()
    elif historicColumn in dfLinkHistoric.columns:
        dfPoint = pd.concat([
            dfLinkHistoric.rename(columns={ historicColumn: vehicleCountName })[vehicleCountName],
            dfPoint[vehicleCountName]
        ]).to_frame()
    
    # Underlying data is 4 minute blocks, make it 8 minutes
    pointDataStart = np.min(dfPoint.index)
    dfPoint = dfPoint.resample('480s', kind='timestamp', base=0).sum(min_count=1)
    gc.collect()
    
    if dfPointTs is None:
        dfPointTs = dfPoint
    else:
        dfPointTs = dfPointTs.join(
            dfPoint, 
            how='outer',
            rsuffix=' (%s)' % systemCodeNumber
        )
        dfPoint = None
    
    gc.collect()
    
dfPointTs.sort_index(inplace=True)

In [ ]:
dfPointTs.tail(10)

In [ ]:
# Deduplicate the index
dfPointTs = dfPointTs.loc[~dfPointTs.index.duplicated(keep='last')]
dfPointTs

In [ ]:
dfPointTs.to_pickle('../cache/recent-traffic-volumes-pd.pkl')
anprLinks.to_pickle('../cache/recent-traffic-volumes-link-metadata-pd.pkl')
anprPoints.to_pickle('../cache/recent-traffic-volumes-point-metadata-pd.pkl')
pdAnprDuplicateRegister.to_pickle('../cache/recent-traffic-volumes-point-alternative-ids.pkl')

In [ ]:
# Sorry folks, but it's an >80MB file otherwise
dfPointTs.resample('960s').sum().to_csv('../output/t&w-anpr-volumes-pd-16min.csv')
anprLinks.to_csv('../output/t&w-anpr-volumes-link-metadata-pd.csv')
anprPoints.to_csv('../output/t&w-anpr-volumes-point-metadata-pd.csv')
pdAnprDuplicateRegister.to_csv('../output/t&w-anpr-volumes-point-metadata-original-id-mapping.csv')